In [1]:
import torch, torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [2]:
batch_size_train = 32 
batch_size_test = 10000
valid_size = 0.2

In [3]:
train_data = datasets.MNIST(root='data', train=True, download=True, transform=ToTensor())
test_data = datasets.MNIST(root='data', train=False, download=True, transform=ToTensor())

In [4]:
print(train_data)
print(test_data)
print(train_data.data.size())
print(test_data.data.size())

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()
torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


In [5]:
def choose_batch(batch_size_trn, batch_size_tst):
    train_loader = DataLoader(train_data, batch_size=batch_size_trn, shuffle=True)
    test_loader = DataLoader(train_data, batch_size=batch_size_tst, shuffle=True)

    return train_loader, test_loader


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_layers = nn.Sequential(nn.Conv2d(1, 10, kernel_size=5), 
                                         nn.MaxPool2d(2), 
                                         nn.ReLU(), 
                                         nn.Conv2d(10, 20, kernel_size=5), 
                                         nn.MaxPool2d(2),
                                         nn.ReLU(),)
        self.fc_layers = nn.Sequential(nn.Linear(320,  50),
                                       nn.ReLU(),
                                       nn.Linear(50, 10),
                                       nn.Softmax(dim = 1))
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        out = self.fc_layers(x)
        return out

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_indx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output.log(), target)
        loss.backward()
        optimizer.step()
        if batch_indx % 100 == 0:
            print('Epoch {0}  Loss {1} '.format(epoch, loss.item()))

In [8]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output.log(), target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    correct /= len(test_loader.dataset)
    print('Avg loss {0} Avg acc {1}'.format(test_loss, correct))

In [9]:
batch_size = 128
epochs = 3
device = torch.device('cpu')

In [10]:
train_loader = DataLoader(datasets.MNIST('mnist_data', train = True, download=True, transform=transforms.Compose([transforms.ToTensor()])), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(datasets.MNIST('mnist_data', train = False, download=True, transform=transforms.Compose([transforms.ToTensor()])), batch_size=batch_size, shuffle=True)

0.7%

100.0%


Extracting mnist_data\MNIST\raw\train-images-idx3-ubyte.gz to mnist_data\MNIST\raw



100.0%
2.0%

Extracting mnist_data\MNIST\raw\train-labels-idx1-ubyte.gz to mnist_data\MNIST\raw



100.0%


Extracting mnist_data\MNIST\raw\t10k-images-idx3-ubyte.gz to mnist_data\MNIST\raw



100.0%


Extracting mnist_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to mnist_data\MNIST\raw



In [11]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [12]:
for epoch in range(1, epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch 1  Loss 2.2937233448028564 
Epoch 1  Loss 2.280858278274536 
Epoch 1  Loss 2.2273497581481934 
Epoch 1  Loss 1.3957494497299194 
Epoch 1  Loss 0.42429718375205994 
Avg loss 0.003208623141050339 Avg acc 0.8798
Epoch 2  Loss 0.46712321043014526 
Epoch 2  Loss 0.4227161705493927 
Epoch 2  Loss 0.25217685103416443 
Epoch 2  Loss 0.3823547661304474 
Epoch 2  Loss 0.17029082775115967 
Avg loss 0.0017711097732186318 Avg acc 0.9318
Epoch 3  Loss 0.30310171842575073 
Epoch 3  Loss 0.26089003682136536 
Epoch 3  Loss 0.12741996347904205 
Epoch 3  Loss 0.234884575009346 
Epoch 3  Loss 0.10671350359916687 
Avg loss 0.0012067803271114826 Avg acc 0.9529
